# project ai: Easer

by Michiel Téblick and thibaut Van Goethem

In this notebook we will look at the easer model proposed at https://dl.acm.org/doi/pdf/10.1145/3308558.3313710.

This model will be applied to a dataset from foods.com which containes a bunch of recipes with user ratings/reactions on them.

In [74]:
import math
from copy import deepcopy
import pandas as pd
import numpy as np
import scipy
import bottleneck as bn
from sklearn.model_selection import KFold
import time
import pickle
from scipy import sparse
import statistics as st

## Reading and preprocessing the data

In [75]:
use_less_data = False


df_train = pd.read_csv('../folds/fold_0/train.csv')
df_test = pd.read_csv('../folds/fold_0/test.csv')
df_validate = pd.read_csv('../folds/fold_0/validate.csv')
df = pd.concat([df_train, df_test, df_validate])
#df.reset_index()

print("amount of interactions in the full dataset: ",len(df))
print("amount of recipes in the full dataset: ",len(df.recipe_id.unique()))
print("amount of users in the full dataset: ",len(df.user_id.unique()))

if use_less_data:
    df = df[df['count_item'] >= 10]
    print("amount of recipes in the smaller dataset: ",len(df.recipe_id.unique()))
    print("amount of users in the smaller dataset: ",len(df.user_id.unique()))
df.reset_index()


amount of interactions in the full dataset:  733951
amount of recipes in the full dataset:  80511
amount of users in the full dataset:  32635


,index,user_id,recipe_id,date,rating,review,count_user,count_item
0,0,56680,79222,2006-11-11,5,"Oh, This was wonderful! Had a soup and salad ...",174,18
1,1,827374,79222,2010-11-29,3,We made this last night and really enjoyed it....,10,18
2,2,462571,208980,2007-07-05,5,These were a snap to whip up and were fantasti...,87,8
3,3,222139,208980,2007-09-08,5,I chose this recipe from Fall Pick A Chef. Co...,446,8
4,4,423539,342209,2009-06-02,5,These are lovely cookies not bland at all. \r\...,35,8
...,...,...,...,...,...,...,...,...
733946,151908,96177,196735,2009-01-12,5,We just loved these tatters. Quick easy and ve...,563,17
733947,151909,573325,196735,2010-09-08,5,"What a great, healthy, easy and yummy recipe!<...",880,17
733948,151910,203111,213546,2007-06-28,5,Very good potatoes! I served them with fried ...,217,6
733949,151911,41468,82303,2006-09-01,5,WOW this was great. What I love the most is th...,30,13


Set all ratings to 1 (even negative interactions are seen as interactions)

In [76]:
df.loc[:,'rating'] = 1

Below here are two ways to cut down on the amount of interactions that are used in this notebook
- The first one randomly removes x% of the users,
- The second one removes all user and recipes that have less than X amount of interaction containing them

We opted for the second form as this is more representative of how the models should be used due to the lower amount if recipes but more reactions per recipe. Also the second choice is a deterministic way of removing data, which the first one is not.
This does end up mostly giving slightly worse result compared to the first choice.

The reason we need to remove data is because a matrix inversion is done, which can not be done in a smart way.
Also the result of the inversion is not necessarily a sparse matrix so the full calculation needs to be done on dense matrices. This end up scaling O(n^3) in time complexity and O(n^2) for memory needed. n here is the amount of recipes.
So running on the full dataset would require more than 200gb of ram which we do not have.


### rescaling the id's
The recipes and users don't go from 0 to amount so if we were to put this in a matrix we would get empty columns and rows. This is not that handy so we reindex both the user_id and recipe_ids

This is a step we must not forget when entering the data in the model, as we also need to remap our input data using the same remapping that was used here

In [77]:
userSet = set(df['user_id'].to_list())
user_transform_dict = dict(map(reversed, enumerate(userSet)))
recipeSet = set(df['recipe_id'].to_list())
recipe_transform_dict = dict(map(reversed, enumerate(recipeSet)))
recipe_dict = dict(enumerate(recipeSet))

In [78]:
keep_nan_user = [k for k, v in user_transform_dict.items() if pd.isnull(v)]
keep_nan_recipe = [k for k, v in recipe_transform_dict.items() if pd.isnull(v)]


def transform_id(dataframe):
    tochange = dataframe['user_id']
    dataframe['user_id'] = tochange.map(user_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan_user)))

    tochange = dataframe['recipe_id']
    dataframe['recipe_id'] = tochange.map(recipe_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan_recipe)))
    return dataframe

def open_csv(filename, use_less_data=False):
    csv_df = pd.read_csv(filename)
    if use_less_data:
        csv_df = csv_df[csv_df['count_item'] >= 10]
    csv_df = transform_id(csv_df)
    csv_df.loc[:,'rating'] = 1
    csv_df.drop('review', axis=1, inplace=True)
    #df.drop('date', axis=1, inplace=True)
    return csv_df


### Creation of the folds


In [79]:
k = 10
folds = list()
for directory in ["../folds/fold_%d" % i for i in range(k)]:
    folds.append((directory + "/train.csv", directory + "/validate.csv",directory + "/test.csv"))

## Creation model
Here we define the models used for the experiments. Both the easer predictor and a populaliry predictor are created. the popularity predictor is used as a baseline

In [80]:
def split_test(data_set):
    ground_truth = data_set.sort_values('date').groupby('user_id').tail(1)
    predict = pd.concat([data_set, ground_truth]).drop_duplicates(keep=False)
    return predict, ground_truth

def data_frame_to_matrix(dataframe):
    ratings = dataframe.rating
    idx = (dataframe.user_id, dataframe.recipe_id)
    return sparse.csc_matrix((ratings, idx), shape=(len(df.user_id.unique()), len(df.recipe_id.unique())),
                                dtype=float)

## Creation model
Here we define the models used for the experiments. Both the easer predictor and a populaliry predictor are created. the popularity predictor is used as a baseline
This code is heavily inspired from: https://github.com/hasteck/MRF_NeurIPS_2019/blob/master/mrf_for_cf_NeurIPS_2019.ipynb
The original code still used a lot of dense representations, we changed this to make more use out of sparse representations.

In [81]:
alpha = 0.75

def precompute_sparse(train_data):
    # this function standardizes the gram matrix, this helps us to later set a threshold value for calculating the sparsity pattern.
    # If the vectors of the gram matrix our centered around 0 then gram matrix is proportional with the covariance matrix.
    # we there for have to center our vectors around 0 and rescale them.
    userCount = train_data.shape[0]
    XtX= train_data.T.dot(train_data) #  Build the data gram matrix X^TX

    # XtX
    mu=XtX.diagonal() / userCount
    variance_times_userCount = XtX.diagonal() - mu * mu * userCount

    # standardizing the data-matrix XtX (if alpha=1, then XtX becomes the correlation matrix)
    XtX -= scipy.sparse.csr_matrix(mu[:,None]) * scipy.sparse.csr_matrix(mu* userCount)
    rescaling = np.power(variance_times_userCount, alpha / 2.0)
    scaling = 1.0  / rescaling

    XtX = scipy.sparse.diags(scaling) * XtX * scipy.sparse.diags(scaling)

    XtXdiag = deepcopy(XtX.diagonal())
    ii_diag = np.diag_indices(XtX.shape[0])

    return XtX,XtXdiag,ii_diag,rescaling,scaling

In [82]:
class sparseMRF:
    def __init__(self,xtx,xtxdiag,ii,rescaling,scaling):
        self.XtX=xtx
        self.XtXdiag = xtxdiag
        self.ii_diag = ii
        self.threshold = 0.375    # results in sparsity 0.1 % (for alpha=0.75)
        #threshold = 0.11    # results in sparsity 0.5 % (for alpha=0.75)
        self.maxInColumn = 1000
        # hyper-parameter r in the paper, which determines the trade-off between approximation-accuracy and training-time
        self.rr = 0.5
        # L2 norm regularization
        self.L2reg = 1.0
        self.scaling=scaling
        self.rescaling=rescaling

    def sparse_solution(self):

        # sparsity pattern, see section 3.1 in the paper
        # self.XtX[self.ii_diag] = self.XtXdiag
        sparsity_pattern = self.calculate_sparsity_pattern()

        # parameter-estimation, see section 3.2 in the paper
        self.XtX[self.ii_diag] = self.XtXdiag+self.L2reg
        BBsparse = self.sparse_parameter_estimation(sparsity_pattern)

        return BBsparse


    def calculate_sparsity_pattern(self):

        # apply threshold
        idx1, idx2, value = sparse.find(np.abs(self.XtX > self.threshold))
        boolean_matrix = scipy.sparse.csc_matrix((value, (idx1, idx2)), dtype=np.float32)
        sparsity_pattern = boolean_matrix.multiply(self.XtX)
        sparsity_pattern.eliminate_zeros()

        # enforce maxInColumn
        countInColumns = sparsity_pattern.getnnz(axis=0) # count number of nonzero values in each column
        columns = np.where(countInColumns > self.maxInColumn)[0] # get all column with more values than maxInColumn
        for i in columns:
            j= sparsity_pattern[:,i].nonzero()[0]
            k = bn.argpartition(-np.abs(np.asarray(sparsity_pattern[j,i].todense()).flatten()), self.maxInColumn)[self.maxInColumn:]
            sparsity_pattern[j[k], i] = 0.0 # set the lowest values to 0
        sparsity_pattern.eliminate_zeros()

        return sparsity_pattern

    def sparse_parameter_estimation(self, sparsity_pattern):
        # this implements section 3.2 in the paper

        # list L in the paper, sorted by item-counts per column, ties broken by item-popularities as reflected by np.diag(XtX)
        CountInColumns = sparsity_pattern.getnnz(axis=0)
        # CountInColumns consists out of integers, so to break the tie on item-popularity we add a value
        # between 0 and 1 to each value in CountInColumns. This value is proportional to the popularity.
        sortedList=np.argsort(CountInColumns+ self.XtX.diagonal() /2.0/ np.max(self.XtX.diagonal()))[::-1]

        # print("iterating through steps 1,2, and 4 in section 3.2 of the paper ...")
        todoIndicators=np.ones(CountInColumns.shape[0]) # structure to keep track of processed columns
        blockList=[]   # list of blocks. Each block is a list of item-indices, to be processed in step 3 of the paper
        for i in sortedList:
            if todoIndicators[i]==1: # if column not yet processed
                n, _, vals=sparse.find(sparsity_pattern[:,i])  # step 1 in paper: set n contains item i and its neighbors N
                k=np.argsort(np.abs(vals))[::-1]
                n=n[k] # sort nn
                blockList.append(n) # list of items in the block, to be processed in step 3 below
                # remove possibly several items from list L, as determined by parameter rr (r in the paper)
                dd_count=max(1,int(np.ceil(len(n)*self.rr)))
                dd=n[:dd_count] # set D, see step 2 in the paper
                todoIndicators[dd]=0  # Set this column to done

        # print("now step 3 in section 3.2 of the paper: iterating ...")
        # now the (possibly heavy) computations of step 3:
        BBlist_ix1, BBlist_ix2, BBlist_val = [], [], []
        denseXtX = self.XtX.toarray()

        # calculate solution for each block matrix
        for n in blockList:
            BBblock=np.linalg.inv(denseXtX[np.ix_(n,n)])
            BBblock/=-np.diag(BBblock)
            # determine set D based on parameter rr (r in the paper)
            dd_count=max(1,int(np.ceil(len(n)*self.rr)))
            dd=n[:dd_count] # set D in paper
            # store the solution regarding the items in D
            blockix = np.meshgrid(dd,n)
            BBlist_ix1.extend(blockix[1].flatten().tolist())
            BBlist_ix2.extend(blockix[0].flatten().tolist())
            BBlist_val.extend(BBblock[:,:dd_count].flatten().tolist())

        del denseXtX

        # Combine all block matrices
        BBsum = sparse.csc_matrix((BBlist_val, (BBlist_ix1, BBlist_ix2)), shape=self.XtX.shape, dtype=np.float32)
        BBcnt = sparse.csc_matrix((np.ones(len(BBlist_ix1), dtype=np.float32), (BBlist_ix1,BBlist_ix2)), shape=self.XtX.shape, dtype=np.float32)
        b_div= sparse.find(BBcnt)[2]
        b_3= sparse.find(BBsum)
        BBavg = sparse.csc_matrix(( b_3[2] / b_div, (b_3[0],b_3[1])), shape=self.XtX.shape, dtype=np.float32)
        BBavg[self.ii_diag]=0.0

        # forcing the sparsity pattern of AA onto BB
        BBavg = sparse.csr_matrix((np.asarray(BBavg[sparsity_pattern.nonzero()]).flatten(),
                                   sparsity_pattern.nonzero()), shape=BBavg.shape, dtype=np.float32)

        return BBavg

    def train(self):
        BBsparse = self.sparse_solution()
        #
        # rescale items back to original popularity
        BBsparse=sparse.diags(scaling).dot(BBsparse).dot(sparse.diags(rescaling))
        self.B = BBsparse

    def predict(self,predict_matrix):
        return predict_matrix.dot(self.B)

## training models + k-fold validation

In [83]:
K = 20
K2 = 50

def recal_easer(model, predict_data, test_data):
    total = len(test_data)

    X_train = data_frame_to_matrix(predict_data)
    y_pred = model.predict(X_train).toarray()

    X_test = data_frame_to_matrix(test_data)

    interacted_recipes = (X_train == 1).toarray()
    y_pred[interacted_recipes] = -100000
    idx_top_scores = np.asarray((-y_pred).argsort()[:,:100])
    del y_pred
    dense_X_test = X_test.toarray()

    correct_K = 0
    correct_K2 = 0
    ndcg = 0

    for idx, row in enumerate(idx_top_scores):
        for rank, index in enumerate(row):
            if dense_X_test[idx][index] == 1:
                if rank < K:
                    correct_K += 1
                if rank < K2:
                    correct_K2 += 1
                ndcg += 1/(math.log2(rank+2))

    print("easer recall@%s = %s" % (str(K), str(correct_K / total)))
    print("easer recall@%s = %s" % (str(K2), str(correct_K2 / total)))
    print("easer ndcg@%s = %s" % (100, str(ndcg / total)), end="\n\n")

    return correct_K/total, correct_K2/total, ndcg/total


## training models + k-fold validation


In [84]:
#Please enter the path here of where you will place the pickle files (with trailing /)
data_path="D:/results_aiproject_improvement/"

In [ ]:
for f_idx, fold_files in enumerate(folds):
    start = time.time()
    train_data = open_csv(fold_files[0], True)
    #Here we have the user item matrix
    X_train = data_frame_to_matrix(train_data)

    #train models

    # model_pop=popularity()
    # model_pop.train(train_data)
    # modelpopfile = open(data_path+"model_pop_fold" + str(f_idx) + ".pkl", mode='wb')
    # pickle.dump(model_pop, modelpopfile)
    # modelpopfile.close()
    # model_pop = None

    highest_recall = (0,0,0)
    best_lambda = 0
    validate_data = open_csv(fold_files[1], use_less_data)
    newstart=start
    for l in range(100, 1500, 100):
        xtx,XtXdiag,ii_diag,rescaling,scaling=precompute_sparse(X_train)
        model=sparseMRF(xtx,XtXdiag,ii_diag,rescaling,scaling)
        model.train()
        print("done training parameter: ", l)
        interactions, ground_truth = split_test(validate_data)

        modelfile = open(data_path+"model_fold" + str(f_idx) + ".pkl", mode='wb')
        pickle.dump(model, modelfile)
        modelfile.close()

        recall20, recall50, ndcg = recal_easer(model, interactions, ground_truth)
        if recall20 > highest_recall[0]:
            highest_recall = (recall20, recall50, ndcg)
            best_lambda = l

            modelfile = open(data_path+"model_fold" + str(f_idx) + ".pkl", mode='wb')
            pickle.dump(model, modelfile)
            modelfile.close()

        end = time.time()
        print("fold took : ", end - newstart, "s")
        newstart=end

    print("done fold:",str(f_idx))

    print("Best parameter lambda: ", best_lambda)
    print("easer fold: %s, recall@%s = %s" % (str(f_idx), str(K), highest_recall[0]))
    print("easer fold: %s, recall@%s = %s" % (str(f_idx), str(K2), highest_recall[1]))
    print("easer fold: %s, ndcg@%s = %s" % (str(f_idx), 100, highest_recall[2]), end="\n\n")

    end = time.time()
    print("training took : ", end - start, "s")
    break



<ipython-input-81-30f462b8f82b>:34: RuntimeWarning: divide by zero encountered in true_divide
  scaling = 1.0  / rescaling
<ipython-input-81-30f462b8f82b>:13: RuntimeWarning: divide by zero encountered in true_divide
  scaling = 1.0  / rescaling
<ipython-input-81-30f462b8f82b>:14: RuntimeWarning: invalid value encountered in multiply
  XtX = scaling[:,None] * XtX * scaling
C:\Users\thiba\anaconda3\lib\site-packages\scipy\sparse\_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
C:\Users\thiba\anaconda3\lib\site-packages\scipy\sparse\_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


done training parameter:  100
easer recall@20 = 0.014399509803921568
easer recall@50 = 0.021752450980392156
easer ndcg@100 = 0.008622092073550386

fold took :  871.6244850158691 s


## Evaluation results of the folds

Here we use recall@20, recal@50 and ndcg@100


In [ ]:
result_list_K = list()
result_list_K2 = list()
result_ndcg = list()

for i in range(k):

    #Evaluate recall@k

    data = open_csv(folds[i][2])
    model = pickle.load(open(data_path+"model_fold"+str(i)+".pkl", mode='rb'))
    interactions, ground_truth = split_test(data)

    recall20, recall50, ndcg = recal_easer(model, interactions, ground_truth)

    result_list_K.append(recall20)
    result_list_K2.append(recall50)
    result_ndcg.append(ndcg)

    print("easer fold: %s, recall@%s = %s" % (str(i), str(K), recall20))
    print("easer fold: %s, recall@%s = %s" % (str(i), str(K2), recall50))
    print("easer fold: %s, ndcg@%s = %s" % (str(i), 100, ndcg), end="\n\n")

print("mean recall@%s over 10 folds: " % str(K), str(st.mean(result_list_K)))
print("mean recall@%s over 10 folds: " % str(K2), str(st.mean(result_list_K2)))
print("mean ndcg@%s over 10 folds: " % str(100), str(st.mean(result_ndcg)), end="\n\n")
print("standard deviation recall@%s over 10 folds: " % str(K), str(st.pstdev(result_list_K)))
print("standard deviation recall@%s over 10 folds: " % str(K2), str(st.pstdev(result_list_K2)))
print("standard deviation ndcg@%s over 10 folds: " % str(100), str(st.pstdev(result_ndcg)))

In [ ]:
#recall score for popularity
result_list_pop_K=list()
result_list_pop_K2=list()
result_list_pop_ndcg=list()
for i in range(k):
    data =  open_csv(folds[i][2])
    model = pickle.load(open(data_path+"model_pop_fold"+str(i)+".pkl", mode='rb'))
    test_data = data[1]
    predict_data = data[0]
    pop=model.predict()
    total = 0
    correct_K = 0
    correct_K2 = 0
    ndcg = 0
    for idx, interaction in test_data.iterrows():
        user = interaction['user_id']
        user_data = predict_data.loc[(predict_data['user_id'] == user)]
        already_interacted_recipes = user_data[user_data.columns[1]].to_numpy()
        newpop = pop[:150]
        newpop = newpop[~np.in1d(newpop,already_interacted_recipes)]
        newpop_K = newpop[:K]
        newpop_K2 = newpop[:K2]
        newpop_ndcg = newpop[:100]
        recipe = interaction['recipe_id']
        if recipe in newpop_K:
            correct_K += 1
        if recipe in newpop_K2:
            correct_K2 += 1
        if recipe in newpop_ndcg:
            ndcg += 1/(math.log2(np.where(newpop_ndcg == recipe)[0]+2))
        total += 1
    result_list_pop_K.append(correct_K / total)
    result_list_pop_K2.append(correct_K2 / total)
    result_list_pop_ndcg.append(ndcg / total)
    print("popularity fold: %s, recall@%s = %s" % (str(i),str(K), str(correct_K / total)))
    print("popularity fold: %s, recall@%s = %s" % (str(i),str(K2), str(correct_K2 / total)))
    print("popularity fold: %s, ndcg@%s = %s" % (str(i),str(100), str(ndcg / total)), end="\n\n")

print("mean recall@%s over 10 folds: " % str(K), str(st.mean(result_list_pop_K)))
print("mean recall@%s over 10 folds: " % str(K2), str(st.mean(result_list_pop_K2)))
print("mean ndcg@%s over 10 folds: " % str(100), str(st.mean(result_list_pop_ndcg)), end="\n\n")
print("standard deviation recall@%s over 10 folds: " % str(K), str(st.pstdev(result_list_pop_K)))
print("standard deviation recall@%s over 10 folds: " % str(K2), str(st.pstdev(result_list_pop_K2)))
print("standard deviation ndcg@%s over 10 folds: " % str(100), str(st.pstdev(result_list_pop_ndcg)))

The next section is a demonstration that selects a random user and makes a recommendation prediction for this user.

In [ ]:
import random
# read recipe data and load pre-trained model
df_recipes = pd.read_csv('../data/RAW_recipes.csv')
df_recipes.drop(['minutes', 'contributor_id', 'submitted', 'tags',
                 'nutrition', 'n_steps', 'steps', 'description', 'n_ingredients'], axis=1, inplace=True)
data = pickle.load(open(data_path+"data_fold0.pkl", mode='rb'))
model = pickle.load(open(data_path+"model_fold0.pkl", mode='rb'))
predict_data = data[0]
ratings = predict_data.rating
idx = (predict_data.user_id, predict_data.recipe_id)
x_train = sparse.csc_matrix((ratings, idx), shape=(len(df.user_id.unique()), len(df.recipe_id.unique())), dtype=float)

# get random user and make prediction
random_user = x_train.getrow(random.randint(0, len(df.user_id.unique())))
prediction = model.predict(random_user)[0]
interacted_recipes = []
for recipe_id in random_user.indices:
    interacted_recipes.append(recipe_dict[recipe_id])
    prediction[recipe_id] = -100000


top_index = (-prediction).argsort()[:10]
recommended_recipes = []
for recipe_id in top_index:
    recommended_recipes.append(recipe_dict[recipe_id])

# get interacted recipes and recommended recipes
user_interactions = df_recipes[df_recipes['id'].isin(interacted_recipes)].drop('id', axis=1)
user_recommendations = df_recipes[df_recipes['id'].isin(recommended_recipes)].drop('id', axis=1)

display(user_interactions)
display(user_recommendations)
